# OD Data

In [1]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

# Downloading the CA OD Data

In [2]:
path = r'R:/DPOE/LEHD LODES/7.5_2021_11Nov/Source/LODES_Download_2021-11-19-20-07-45/od'

# 2019 source data
csv_2019 = glob.glob(path + "/*2019.csv")
list_2019 = []

for filename in csv_2019:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_2019.append(df)

od_2019 = pd.concat(list_2019, axis=0, ignore_index=True)

od_2019


,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
0,60014001001024,484391108072031,1,1,0,0,0,0,1,0,0,1,20211018
1,60014002001003,80050049522007,1,0,1,0,0,1,0,0,0,1,20211018
2,60014002001018,410470016044008,1,0,1,0,0,0,1,0,0,1,20211018
3,60014002001020,481350008004038,1,0,1,0,1,0,0,0,0,1,20211018
4,60014002001022,80310009035004,1,0,1,0,0,1,0,0,0,1,20211018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57374695,61150401005037,61150407001024,1,0,1,0,0,0,1,0,0,1,20211018
57374696,61150401005057,60670095041003,1,0,1,0,0,0,1,0,0,1,20211018
57374697,61150401005057,61150406005000,1,0,1,0,0,0,1,0,0,1,20211018
57374698,61150401005067,61130113003034,1,1,0,0,0,0,1,0,0,1,20211018


# Loading the staging table (This is all of CA and more)

In [3]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=dpoe_stage;'
                      'Trusted_Connection=yes;')

In [4]:
od_db19 ="SELECT * "\
                  "FROM [dpoe_stage].[lehd_lodes].[od_7_5_20211018]"\
                   "WHERE yr= 2019"
db19 = pd.read_sql_query(od_db19, conn)


In [ ]:
db19

,w_geoid,h_geoid,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,type,yr
0,060730085111105,060710099101004,1,0,1,0,0,1,0,0,0,1,JT00,2019
1,060730085111105,060730009006011,1,0,1,0,0,1,0,0,0,1,JT00,2019
2,060730085111105,060730018001005,1,0,1,0,0,1,0,0,0,1,JT00,2019
3,060730085111105,060730030032007,1,0,1,0,0,1,0,0,0,1,JT00,2019
4,060730085111105,060730030042004,1,0,1,0,0,1,0,0,0,1,JT00,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5763999,060730190013013,060730190012058,3,0,3,0,0,0,3,3,0,0,JT03,2019
5764000,060730200172022,060730200202007,3,0,3,0,0,0,3,3,0,0,JT03,2019
5764001,060730085111133,060730170513000,3,0,3,0,0,0,3,3,0,0,JT02,2019
5764002,060730085111133,060730170291074,3,0,3,0,0,0,3,3,0,0,JT03,2019


# Compare the OD Data

In [6]:
# Subsetting the source data (Subsetting the CA Data)
od_2019_sub_wgeocode = od_2019[od_2019['w_geocode'].astype(str).str.startswith('6073')]

In [7]:
od_2019_sub_hgeocode = od_2019[od_2019['h_geocode'].astype(str).str.startswith('6073')]

In [8]:
od_2019_sub_both_CA = od_2019[(od_2019['w_geocode'].astype(str).str.startswith('6073')) & (od_2019['h_geocode'].astype(str).str.startswith('6073'))]

In [9]:
w_ca_h_other = od_2019[(od_2019['w_geocode'].astype(str).str.startswith('6073')) & ~(od_2019['h_geocode'].astype(str).str.startswith('6073'))]

In [5]:
w_other_h_ca = od_2019[~(od_2019['w_geocode'].astype(str).str.startswith('6073')) & (od_2019['h_geocode'].astype(str).str.startswith('6073'))]

In [14]:
w_or_g_is_CA = od_2019[(od_2019['w_geocode'].astype(str).str.startswith('6073')) | (od_2019['h_geocode'].astype(str).str.startswith('6073'))]

In [16]:
od_2019_sub= od_2019 [od_2019 ['w_geocode'].astype(str).str.startswith('6073') | od_2019 ['h_geocode'].astype(str).str.startswith('6073') ]

In [18]:
neither = od_2019 [~(od_2019 ['w_geocode'].astype(str).str.startswith('6073')) | ~(od_2019 ['h_geocode'].astype(str).str.startswith('6073')) ]

# Comparing Lengths

In [15]:
# print(f"The length of the raw OD data is {len(od_2019)}")
# print(f"The length of the OD data when w_geocode is filtered for 6073 is {len(od_2019_sub_wgeocode)}")
# print(f"The length of the OD data when h_geocode is filtered for 6073 is {len(od_2019_sub_hgeocode)}")
# print(f"The length of the OD data when both g_geocode and h_geocode is filtered for 6073 is {len(od_2019_sub_both_CA)}")
# print(f"The length of the data when w_geocode is 6073 but h_geocode is not {len(w_ca_h_other)}")
print(f"The length of the data when h_geocode is 6073 but w_geocode is not {len(w_other_h_ca)}")
print(f"The length of the data when either h_geocode or w_geocode is 6073 {len(w_or_g_is_CA)}")

The length of the data when h_geocode is 6073 but w_geocode is not 1004754
The length of the data when either h_geocode or w_geocode is 6073 5713340


In [ ]:
print(f"2019 CA Data Length: {len(od_2019_sub)}")
print(f"2019 Staging Data Length: {len(db19)}")

2019 CA Data Length: 4708586
2019 Staging Data Length: 5764004
